In [2]:
%pip install --upgrade --quiet  langchain langchain-openai langchain-experimental presidio-analyzer presidio-anonymizer spacy Faker




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 638.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1

In [3]:
!python -m spacy download en_core_web_lg



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
#simple anonymizer
from langchain_experimental.data_anonymizer import PresidioAnonymizer

anonymizer = PresidioAnonymizer()

anonymizer.anonymize(
    "My name is zayd amghar, call me at 0657196723 or email me at real.slim.shady@gmail.com"
)

'My name is Jacob White amghar, call me at 001-584-482-7657x567 or email me at jasonsmith@example.org'

In [11]:
#simple anonymier
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON", "PHONE_NUMBER","EMAIL_ADDRESS","CREDIT_CARD"])
anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com and my credit card with the number 4916 0387 9536 0861. "
)

'My name is Cassie Thomas amghar, call me at 226-428-6474x998 or email me at Cassie Thomasamghar5@gmail.com and my credit card with the number 5387317336586979. '

In [49]:
#pattern identiifire
from presidio_analyzer import Pattern, PatternRecognizer

id_Pattern = Pattern(
    name="id_Pattern",
    regex="/d{11}",
    score=1,
)


id_recognizer = PatternRecognizer(
    supported_entity="ID", patterns=[id_Pattern]
)

anonymizer.add_recognizer(id_recognizer)

In [50]:
#anonimyzer fields missing the ssn
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON", "PHONE_NUMBER","EMAIL_ADDRESS","CREDIT_CARD","ID"])
anonymizer.anonymize(
        "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com and my credit card with the number 4916 0387 9536 0861 and My id number is 95525987444 "
)

'My name is Nathan Howard, call me at 401-810-4477 or email me at qlam@example.net and my credit card with the number 213126599775273 and My id number is 95525987444 '

In [51]:
#create synthetic data for the new idendeties
from faker import Faker

fake = Faker(locale="pl_PL")


def fake_id_number(_=None):
    return fake.ssn()


fake_id_number()

'98070407382'

In [52]:
from presidio_anonymizer.entities import OperatorConfig

new_operators = {
    "ID": OperatorConfig(
        "custom", {"lambda": fake_id_number}
    )
}
anonymizer.add_operators(new_operators)

In [56]:
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON", "PHONE_NUMBER","EMAIL_ADDRESS","CREDIT_CARD","ID"])
anonymizer.anonymize(
        "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com and my credit card with the number 4916 0387 9536 0861 and My ssn is 95525987444 "
)

'My name is Steven Harding, call me at +1-497-848-6867 or email me at tylerjennifer@example.net and my credit card with the number 4661282370549918259 and My ssn is 95525987444 '